In [22]:
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import sys 
import os
src_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
project_path = os.path.abspath(os.path.join(src_path, os.pardir))
sys.path.insert(1, src_path)
sys.path.insert(1, project_path)

from experiments.experiment_main import prediction_plot, error_plots_flexible, collect_data, L2, H1, models_color_dict
from src.experiments.Proceedings.parameters import epsilons2try
from experiments.Proceedings.parameters import k, alpha
from lib.utils import NamedPartial

import ipywidgets as widgets

data_experiment_name = "Proceedings"

# Visualizing experiments

Explore how each method behaves for different $\varepsilon$, K, float_precision, sampling method. Plots for error as a function of $\varepsilon$; time and accurcy and predictions plots are available.

In [27]:
def show_experiment(experiment_name):
    if experiment_name == "n_train":
        from experiments.Proceedings.n_train import n_train
        from experiments.Proceedings.n_train import float_precision
        from experiments.Proceedings.n_train import sampler
    elif experiment_name == "sampling_method":
        from experiments.Proceedings.sampling_method import n_train
        from experiments.Proceedings.sampling_method import float_precision
        from experiments.Proceedings.sampling_method import sampler
    elif experiment_name == "float_precision":
        from experiments.Proceedings.float_precision import n_train
        from experiments.Proceedings.float_precision import float_precision
        from experiments.Proceedings.float_precision import sampler
    else:
        raise Exception("Only n_train, sampling_method and float_precision experiments are specyfied.")
    
    
    df, predictions, true_solutions = collect_data(data_experiment_name, k=k, alpha=alpha, 
                                                   n_train=n_train, sampler=sampler, float_precision=float_precision)

    def jupyter_plot_computation_time_accuracy(epsilon, error, float_precision, sampler, n_train):
        if len(float_precision) > 1:
            hue = "float_precision"
        if len(sampler) > 1:
            hue = "sampler"
        if len(n_train) > 1:
            hue = "n_train"

        filter_dict = {"epsilon": epsilon}

        fig, ax = plt.subplots(ncols=2, figsize=(14, 6))

        sns.boxenplot(data=df[(df[list(filter_dict)] == pd.Series(filter_dict)).all(axis=1)], y="time", x="model_name",
                      hue=hue, ax=ax[0])
        ax[0].set_yscale("log")
        ax[0].set_title("Computing time")

        sns.boxenplot(data=df[(df[list(filter_dict)] == pd.Series(filter_dict)).all(axis=1)], y=H1, x="model_name",
                      hue=hue, ax=ax[1])
        ax[1].set_yscale("log")
        ax[1].set_title("Accuracy by number of train sampling points.")

        plt.show()

    def jupyter_plot_predictions(epsilon, error, n_train, float_precision, sampler):

        filter_dict = {"epsilon": epsilon, "n_train": n_train, "float_precision":float_precision, "sampler":sampler}
        fig, ax = plt.subplots(figsize=(12, 6))
        prediction_plot(df[(df[list(filter_dict)] == pd.Series(filter_dict)).all(axis=1)], predictions, true_solutions, ax, error, n_train, float_precision, sampler, epsilon)
        plt.show()

    summary_stattistic_functions_dict = {"min": np.min, "max": np.max, "median": np.median, "mean": np.mean} 

    def jupyter_plot_error_curves(statistic, error, n_train, float_precision, sampler):
        aggfunc = summary_stattistic_functions_dict[statistic]
        filter_dict = {"n_train": n_train, "float_precision":float_precision, "sampler":sampler}
        fig, ax = plt.subplots(figsize=(12, 6))
        error_plots_flexible(df[(df[list(filter_dict)] == pd.Series(filter_dict)).all(axis=1)], ax, error_name=error, label_var="model_name", x_var="epsilon",
                             aggfunc=aggfunc, color_dict=models_color_dict, n_train=n_train)
        ax.set_ylim((1e-9, 1))
        plt.show()
    
    widgets.interact(
        jupyter_plot_error_curves,
        statistic=widgets.Dropdown(options=list(summary_stattistic_functions_dict.keys()), description='Summary statistic: '),
        error=widgets.Dropdown(options=[L2, H1], description='Error: '),
        n_train=widgets.Dropdown(options=n_train, description='N train: '),
        float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
        sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
        )
    
    widgets.interact(
        jupyter_plot_predictions,
        epsilon=widgets.Dropdown(options=epsilons2try.tolist(), description=r'$\varepsilon$: '),
        error=widgets.Dropdown(options=[L2, H1], description='Error: '),
        n_train=widgets.Dropdown(options=n_train, description='N train: '),
        float_precision=widgets.Dropdown(options=float_precision, description='Float precision: '),
        sampler=widgets.Dropdown(options=sampler, description='Sampling method: ')
        )
    
    widgets.interact(
        jupyter_plot_computation_time_accuracy,
        epsilon=widgets.Dropdown(options=epsilons2try.tolist(), description=r'$\varepsilon$: '),
        error=widgets.Dropdown(options=[L2, H1], description='Error: '),
        float_precision=[float_precision],
        sampler=[sampler],
        n_train=[n_train]
        )
    
widgets.interact(
    show_experiment,
    experiment_name=widgets.Dropdown(options=["n_train", "float_precision", "sampling_method"], description='experiment: ')
    )

interactive(children=(Dropdown(description='experiment: ', options=('n_train', 'float_precision', 'sampling_me…

<function __main__.show_experiment(experiment_name)>